In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the dataset and preprocess it
df = pd.read_csv('NSE-TATAGLOBAL.csv', parse_dates=['Date'], index_col='Date')
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df)

In [3]:
# Create a function to create a stacked LSTM model
def create_model(units, n_features):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(None, n_features)))
    model.add(LSTM(units))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [4]:
# Split the dataset into training and testing sets
train_size = int(len(df_scaled) * 0.8)
train_data = df_scaled[:train_size, :]
test_data = df_scaled[train_size:, :]

In [5]:
# Create sequences and labels for the training data
n_steps = 60
x_train, y_train = [], []
for i in range(n_steps, len(train_data)):
    x_train.append(train_data[i - n_steps:i, :])
    y_train.append(train_data[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)

In [6]:
# Create sequences and labels for the testing data
x_test, y_test = [], []
for i in range(n_steps, len(test_data)):
    x_test.append(test_data[i - n_steps:i, :])
    y_test.append(test_data[i, 0])
x_test, y_test = np.array(x_test), np.array(y_test)

In [7]:
# Build the model and fit it to the training data
model = create_model(50, df_scaled.shape[1])
model.fit(x_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
49/49 [==============================] - 7s 33ms/step - loss: 0.0092
Epoch 2/50
49/49 [==============================] - 2s 33ms/step - loss: 0.0012
Epoch 3/50
49/49 [==============================] - 2s 34ms/step - loss: 0.0010
Epoch 4/50
49/49 [==============================] - 2s 34ms/step - loss: 9.3007e-04
Epoch 5/50
49/49 [==============================] - 2s 34ms/step - loss: 9.4340e-04
Epoch 6/50
49/49 [==============================] - 2s 34ms/step - loss: 8.8985e-04
Epoch 7/50
49/49 [==============================] - 2s 34ms/step - loss: 8.0956e-04
Epoch 8/50
49/49 [==============================] - 2s 34ms/step - loss: 8.1752e-04
Epoch 9/50
49/49 [==============================] - 2s 34ms/step - loss: 6.9021e-04
Epoch 10/50
49/49 [==============================] - 2s 33ms/step - loss: 6.5729e-04
Epoch 11/50
49/49 [==============================] - 2s 34ms/step - loss: 6.5855e-04
Epoch 12/50
49/49 [==============================] - 2s 37ms/step - loss: 6.5688e-04
E

In [8]:
# Evaluate the performance of the model on the testing data
mse = model.evaluate(x_test, y_test)
print("Mean Squared Error: {:.2f}".format(mse))

11/11 [==============================] - 1s 14ms/step - loss: 1.4466e-04
Mean Squared Error: 0.00


In [9]:
# Make predictions on the testing data using the trained model
y_pred = model.predict(x_test)

11/11 [==============================] - 1s 14ms/step


In [10]:
# Inverse scale the predictions and actual values
y_pred = scaler.inverse_transform(np.concatenate((y_pred, x_test[:, -1, 1:]), axis=1))[:, 0]
y_test = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), x_test[:, -1, 1:]), axis=1))[:, 0]

In [11]:
# Print the predicted and actual values
predictions = pd.DataFrame({'Date': df.index[train_size + n_steps:], 'Predicted Close': y_pred, 'Actual Close': y_test})
print(predictions)

          Date  Predicted Close  Actual Close
0   2011-12-09        90.171064         88.75
1   2011-12-08        90.543982         90.85
2   2011-12-07        91.052089         91.00
3   2011-12-05        91.624300         91.70
4   2011-12-02        92.199573         88.60
..         ...              ...           ...
342 2010-07-27       116.884757        117.60
343 2010-07-26       117.837014        120.10
344 2010-07-23       118.927651        121.80
345 2010-07-22       120.459073        120.30
346 2010-07-21       121.092256        122.10

[347 rows x 3 columns]
